In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dep-nlp/dev.csv
/kaggle/input/dep-nlp/train.csv
/kaggle/input/dep-nlp/test.csv
/kaggle/input/dep-nlp-emb/dev-stack-label.npy
/kaggle/input/dep-nlp-emb/dev-stack-emb.npy
/kaggle/input/dep-nlp-emb/train-stack-emb.npy
/kaggle/input/dep-nlp-emb/train-stack-label.npy


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
# from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import tensorflow as tf
print(tf.__version__)

2.6.4


In [3]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [24]:
train = pd.read_csv('../input/dep-nlp/train.csv')
dev = pd.read_csv('../input/dep-nlp/dev.csv')

In [25]:
train.head()

,id,text,label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate


In [26]:
dev.head()

,id,text,label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate
2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate
4,dev_pid_5,The world only cares about beautiful people : ...,moderate


In [27]:
train = train.drop('id', axis=1)
dev = dev.drop('id', axis=1)

In [28]:
frames = [train, dev]
df = pd.concat(frames)

In [29]:
df.head()

,text,label
0,Waiting for my mind to have a breakdown once t...,moderate
1,My new years resolution : I'm gonna get my ass...,moderate
2,New year : Somone else Feeling like 2020 will ...,moderate
3,"My story I guess : Hi, Im from Germany and my ...",moderate
4,Sat in the dark and cried myself going into th...,moderate


In [30]:
df.insert(0, 'id', range(0, len(df)))
df.head()

,id,text,label
0,0,Waiting for my mind to have a breakdown once t...,moderate
1,1,My new years resolution : I'm gonna get my ass...,moderate
2,2,New year : Somone else Feeling like 2020 will ...,moderate
3,3,"My story I guess : Hi, Im from Germany and my ...",moderate
4,4,Sat in the dark and cried myself going into th...,moderate


In [31]:
df.label.unique()

array(['moderate', 'not depression', 'severe'], dtype=object)

In [32]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)
# df['label'] = df.label.replace(label_dict)
# dev_clean['label'] = dev_clean.label.replace(label_dict)


{'moderate': 0, 'not depression': 1, 'severe': 2}


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13387 entries, 0 to 4495
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13387 non-null  int64 
 1   text    13387 non-null  object
 2   label   13387 non-null  object
dtypes: int64(1), object(2)
memory usage: 418.3+ KB


In [34]:
df['text']=df['text'].str.lower()

In [35]:
# lemmatization
def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)

df['text'] = list(map(getLemmText,df['text']))

In [36]:
# stemming
def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)

df['text'] = list(map(getStemmText,df['text']))

In [37]:
# train test split
xtrain, xtest, ytrain, ytest = train_test_split(
 df['text'], df['label'], 
 test_size=0.33, 
 random_state=53)

print(xtrain.shape)
print(xtest.shape)
print(ytrain)

(8969,)
(4418,)
3350    not depression
2636          moderate
7436    not depression
7221    not depression
8046            severe
             ...      
3261          moderate
6565    not depression
2933          moderate
98            moderate
7705    not depression
Name: label, Length: 8969, dtype: object


In [38]:
EMBEDDING_DIMENSION = 64
VOCABULARY_SIZE = 2000
MAX_LENGTH = 100
OOV_TOK = '<OOV>'
TRUNCATE_TYPE = 'post'
PADDING_TYPE = 'post'

In [39]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE, oov_token=OOV_TOK)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [40]:
xtrain_sequences = tokenizer.texts_to_sequences(xtrain)
xtest_sequences = tokenizer.texts_to_sequences(xtest)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))
dict(list(word_index.items())[0:10])

Vocabulary size: 13888


{'<OOV>': 1,
 'i': 2,
 'to': 3,
 'and': 4,
 'a': 5,
 'the': 6,
 'my': 7,
 'it': 8,
 'of': 9,
 'me': 10}

In [41]:
print(xtrain_sequences[100])

[2, 21, 68, 280, 27, 2, 13, 12, 36, 58, 3, 86, 64, 25, 22, 27, 2, 162, 86, 26, 390, 2, 21, 1, 542, 44, 41, 1, 161, 106, 7, 40, 17, 790, 583, 2, 55, 249, 53, 3, 266, 284, 1116, 429, 9, 85, 218, 11, 2, 21, 293, 9, 1, 15, 11, 302, 23, 427, 3, 7, 630, 497, 116, 61, 11, 73, 436, 3, 80, 10, 112, 64, 25, 11, 161, 86, 9, 377, 7, 40, 113, 52, 205, 7, 124, 35, 556, 142, 4, 99, 15, 382, 1, 37, 9, 7, 54, 73, 541, 37, 9, 85, 74, 3, 123, 1, 9, 7, 437, 237, 2, 62, 5, 916, 65, 24, 68, 1556, 4, 99, 15, 65, 86, 565, 10, 161, 611, 239, 65, 467, 50, 26, 10, 7, 1711, 13, 12, 142, 42, 10, 4, 95, 9, 85, 167, 10, 363, 28, 302, 66, 206, 8, 17, 12, 11, 145, 304, 26, 8, 299, 15, 11, 68, 182, 18, 10, 7, 184, 66, 11, 8, 30, 37, 541, 171, 51, 218, 11, 2, 19, 13, 8, 48, 2, 32, 95, 1, 141, 7, 1191, 185, 499, 51, 218, 11, 2, 55, 339, 122, 415, 26, 7, 584, 1, 1276, 2, 55, 249, 62, 476, 185, 499, 37, 9, 85, 134, 50, 26, 34, 852, 41, 119, 124, 4, 54, 388, 68, 142, 11, 2, 75, 23, 339, 51, 178, 87, 487, 9, 38, 3, 1906, 3, 

In [42]:
xtrain_pad = sequence.pad_sequences(xtrain_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
xtest_pad = sequence.pad_sequences(xtest_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
print(len(xtrain_sequences[0]))
print(len(xtrain_pad[0]))
print(xtrain_pad[100])

12
100
[   2   21   68  280   27    2   13   12   36   58    3   86   64   25
   22   27    2  162   86   26  390    2   21    1  542   44   41    1
  161  106    7   40   17  790  583    2   55  249   53    3  266  284
 1116  429    9   85  218   11    2   21  293    9    1   15   11  302
   23  427    3    7  630  497  116   61   11   73  436    3   80   10
  112   64   25   11  161   86    9  377    7   40  113   52  205    7
  124   35  556  142    4   99   15  382    1   37    9    7   54   73
  541   37]


In [43]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(list(ytrain))
training_label_seq = np.array(label_tokenizer.texts_to_sequences(ytrain))
test_label_seq = np.array(label_tokenizer.texts_to_sequences(ytest))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

[2, 3]
[1]
[2, 3]
(8969,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [44]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_article(text):
 return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(xtrain_pad[11]))

i am scare of depress i have been feel late <OOV> depress but not suicid thought just like i am alon and nobodi like me and my friend dont understand me i dont have anyon to vent to and it get tough <OOV> help ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [45]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIMENSION))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(EMBEDDING_DIMENSION, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(EMBEDDING_DIMENSION, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(EMBEDDING_DIMENSION, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

2022-11-08 04:22:53.621633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 04:22:53.715201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 04:22:53.716003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 04:22:53.717424: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [46]:
# training_label_seq

In [47]:
type(xtrain_pad)
# xtrain_pad = np.asarray(xtrain_pad).astype(np.float32)
# xtest_pad = np.asarray(xtest_pad).astype(np.float32)
# training_label_seq = np.asarray(training_label_seq).astype(np.float32)
# test_label_seq = np.asarray(test_label_seq).astype(np.float32)

numpy.ndarray

In [51]:
training_label_seq = np.array([np.array(xi) for xi in training_label_seq])
test_label_seq = np.array([np.array(yi) for yi in test_label_seq])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [53]:
xtrain_pad = tf.convert_to_tensor(xtrain_pad, dtype=tf.int64) 
xtest_pad = tf.convert_to_tensor(xtest_pad, dtype=tf.int64) 
training_label_seq = tf.convert_to_tensor(training_label_seq, dtype=tf.int64) 
test_label_seq = tf.convert_to_tensor(test_label_seq, dtype=tf.int64) 

ValueError: setting an array element with a sequence.

In [49]:
# training_label_seq = np.asarray(training_label_seq).astype(np.float32)
# test_label_seq = np.asarray(test_label_seq).astype(np.float32)

In [50]:
num_epochs = 10
history = model.fit(xtrain_pad, training_label_seq, epochs=num_epochs, validation_data=(xtest_pad, test_label_seq), verbose=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).